In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les données transformées
df = pd.read_csv('../data/processed/pet_adoption_transformed.csv')

# Définir les variables dépendantes (target) et indépendantes (features)
X = df.drop('AdoptionLikelihood', axis=1)  # Variables explicatives (toutes les colonnes sauf 'AdoptionLikelihood')
y = df['AdoptionLikelihood']  # Variable cible (la variable dépendante)

# Variables catégorielles et numériques originales
categorical_columns = ['PetType', 'Breed', 'Color', 'Size', 'Vaccinated', 'HealthCondition', 'PreviousOwner']
numeric_columns = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']

# 1. Division des données (70% pour l'entraînement, 30% pour le test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 3. Initialisation des modèles
models = {
    'Arbre de décision': DecisionTreeClassifier(random_state=42),
    'Forêt aléatoire': RandomForestClassifier(n_estimators=100, random_state=42),
    'k-NN': KNeighborsClassifier(n_neighbors=5)  # Ajout du modèle k-NN
}

# 4. Paramètres pour le GridSearchCV
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 5. Entraînement et évaluation des modèles
results = {}
feature_importances = {}

for model_name, model in models.items():
    print(f"Entraînement du modèle: {model_name}")

    # Si c'est un modèle RandomForest, utiliser GridSearchCV pour l'optimisation des hyperparamètres
    if model_name == 'Forêt aléatoire':
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_  # Meilleur modèle après recherche
        print(f"Meilleurs paramètres pour {model_name}: {grid_search.best_params_}")
    else:
        model.fit(X_train, y_train)

    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Probabilités pour la classe positive (nécessaire pour la ROC et AUC)
    
    # Calculer les métriques de performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    # Stocker les résultats
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'AUC': auc
    }
    
    # Stocker l'importance des caractéristiques
    if hasattr(model, 'feature_importances_'):
        feature_importances[model_name] = model.feature_importances_
    
    # Générer la courbe ROC
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.2f})')

# 6. Afficher les métriques d'évaluation
print("Évaluation des modèles :")
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

# 7. Tracer la courbe ROC
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Ligne de base (chance)
plt.title('Courbe ROC')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc='lower right')
plt.show()

# 8. Analyse des caractéristiques les plus importantes
# Remapper les variables transformées (One-Hot) vers les variables originales
columns_mapping = list(X.columns)  # Nom des variables après le One-Hot Encoding

for model_name, importances in feature_importances.items():
    if importances is not None:
        print(f"\nImportance des caractéristiques pour le modèle {model_name}:")
        
        # Créer un DataFrame avec l'importance des variables
        feature_importance_df = pd.DataFrame({
            'Feature': columns_mapping,
            'Importance': importances
        })
        
        # Trier par importance et afficher les 10 variables les plus importantes
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        print(feature_importance_df.head(10))
        
        # Tracer l'importance des variables
        plt.figure(figsize=(10, 6))
        sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
        plt.title(f'Importance des caractéristiques - {model_name}')
        plt.show()